In [1]:
from ase.build import fcc100, add_adsorbate, molecule
from ase.optimize import LBFGS
from fairchem.core import pretrained_mlip, FAIRChemCalculator

predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="oc20")

# Set up your system as an ASE atoms object
slab = fcc100("Cu", (3, 3, 3), vacuum=8, periodic=True)
slab.calc = calc
print("Before",slab.get_potential_energy())

adsorbate = molecule("CO")
add_adsorbate(slab, adsorbate, 2.0, "bridge")

slab.calc = calc
print("After",slab.get_potential_energy())

# print(slab.get_potential_energy())

# Set up LBFGS dynamics object
opt = LBFGS(slab)
opt.run(0.05, 100)

/home/minkyu/micromamba/envs/fairchem/lib/python3.10/site-packages/torchtnt/utils/version.py:12: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Before -78.92664258939277
After -89.54413964585648
       Step     Time          Energy          fmax
LBFGS:    0 13:44:57      -89.544140       11.491965
LBFGS:    1 13:44:57      -92.462341        6.582732
LBFGS:    2 13:44:57      -92.585171        7.551271
LBFGS:    3 13:44:57      -92.964017        3.752810
LBFGS:    4 13:44:57      -93.123248        3.560934
LBFGS:    5 13:44:57      -93.230204        2.254662
LBFGS:    6 13:44:58      -93.470364        1.136101
LBFGS:    7 13:44:58      -93.561651        0.999685
LBFGS:    8 13:44:58      -93.672328        0.700207
LBFGS:    9 13:44:58      -93.759911        0.502981
LBFGS:   10 13:44:58      -93.806733        0.364669
LBFGS:   11 13:44:58      -93.825594        0.344455
LBFGS:   12 13:44:58      -93.849676        0.485413
LBFGS:   13 13:44:58      -93.868526        0.430901
LBFGS:   14 13:44:58      -93.878750        0.161250
LBFGS:   15 13:44:59      -93.884703        0.168629
LBFGS:   16 13:44:59      -93.891146        0.2045

np.True_

In [15]:
from pymatgen.core import Structure
from ase.visualize import view
from pymatgen.io.ase import AseAtomsAdaptor
from ase import Atoms
from tqdm import tqdm
import pandas as pd
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from io import StringIO
from ase.visualize import view

df = pd.read_csv("/home/minkyu/DiffCSP/data/oc20_dense_small/train.csv") 

diff_energy_list = []

for j in range(20):
  for i in range(11):
    pred_structure = Structure.from_file(f"/home/minkyu/DiffCSP/hydra/singlerun/2025-07-24/oc20_dense_small_CSP_batch_up/eval_diff_train.dir/{j}/cif/{i}.cif")
    pred_slab = AseAtomsAdaptor.get_atoms(pred_structure)

    pred_slab = Atoms(
      symbols=pred_slab.get_chemical_symbols(),
      positions=pred_slab.get_positions(),
      cell=pred_slab.get_cell(),
      pbc=[True, True, True]
    )
    
    # train.csv 경로에 따라 변경
    cif_string = df.loc[i, 'cif']

    # 1. CIF 문자열을 pymatgen Structure로 읽기
    cif_io = StringIO(cif_string)
    true_slab = Structure.from_str(cif_io.read(), fmt="cif")

    # 2. Structure → ASE Atoms 변환
    true_slab = AseAtomsAdaptor.get_atoms(true_slab)
    
    true_slab = Atoms(
      symbols=true_slab.get_chemical_symbols(),
      positions=true_slab.get_positions(),
      cell=true_slab.get_cell(),
      pbc=[True, True, True]
    )
    
    pred_slab.calc = calc
    true_slab.calc = calc
      
    diff_energy = abs(pred_slab.get_potential_energy() - true_slab.get_potential_energy())
    diff_energy_list.append(diff_energy)
  
print(sum(diff_energy_list)/len(diff_energy_list))

# # adsorbate = molecule("CO")
# # add_adsorbate(slab, adsorbate, 2.0, "bridge")

# slab.calc = calc

# print(slab.get_potential_energy())

# view(slab, viewer="ngl")

/home/minkyu/micromamba/envs/fairchem/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 7 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/home/minkyu/micromamba/envs/fairchem/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/home/minkyu/micromamba/envs/fairchem/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/home/minkyu/micromamba/envs/fairchem/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserW

0.42433405247601597


In [ ]:
from pymatgen.core import Structure
from ase.visualize import view
from pymatgen.io.ase import AseAtomsAdaptor
from ase import Atoms
from tqdm import tqdm
import pandas as pd
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from io import StringIO
from ase.visualize import view

df = pd.read_csv("/home/minkyu/DiffCSP/data/oc20_dense_small/train.csv") 

pred_structure = Structure.from_file(f"/home/minkyu/DiffCSP/hydra/singlerun/2025-07-24/oc20_dense_small_CSP_batch_up/eval_diff_train.dir/0/cif/0.cif")
pred_slab = AseAtomsAdaptor.get_atoms(pred_structure)

pred_slab = Atoms(
  symbols=pred_slab.get_chemical_symbols(),
  positions=pred_slab.get_positions(),
  cell=pred_slab.get_cell(),
  pbc=[True, True, True]
)

# train.csv 경로에 따라 변경
cif_string = df.loc[8, 'cif']

# 1. CIF 문자열을 pymatgen Structure로 읽기
cif_io = StringIO(cif_string)
true_slab = Structure.from_str(cif_io.read(), fmt="cif")

# 2. Structure → ASE Atoms 변환
true_slab = AseAtomsAdaptor.get_atoms(true_slab)

true_slab = Atoms(
  symbols=true_slab.get_chemical_symbols(),
  positions=true_slab.get_positions(),
  cell=true_slab.get_cell(),
  pbc=[True, True, True]
)

pred_slab.calc = calc
true_slab.calc = calc

print(pred_slab.get_potential_energy(), true_slab.get_potential_energy(), diff_energy)

In [2]:
from pymatgen.core import Structure
from ase.visualize import view
from pymatgen.io.ase import AseAtomsAdaptor
from ase import Atoms
from tqdm import tqdm
import pandas as pd
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from io import StringIO
from ase.visualize import view
from ase.build import fcc100, add_adsorbate, molecule
from ase.optimize import LBFGS
from fairchem.core import pretrained_mlip, FAIRChemCalculator

predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="oc20")

pred_structure = Structure.from_file(f"/home/minkyu/DiffCSP/original_atoms.cif")
pred_slab = AseAtomsAdaptor.get_atoms(pred_structure)

pred_slab = Atoms(
  symbols=pred_slab.get_chemical_symbols(),
  positions=pred_slab.get_positions(),
  cell=pred_slab.get_cell(),
  pbc=[True, True, True]
)

pred_slab.calc = calc

print(pred_slab.get_potential_energy())

opt = LBFGS(pred_slab)
opt.run(0.01, 100)

-181.56493634303138
       Step     Time          Energy          fmax
LBFGS:    0 00:50:01     -181.564936        0.525506
LBFGS:    1 00:50:01     -181.598345        0.496825
LBFGS:    2 00:50:01     -182.004011        0.636048
LBFGS:    3 00:50:02     -182.020344        0.285235
LBFGS:    4 00:50:02     -182.033215        0.239653
LBFGS:    5 00:50:02     -182.058360        0.531807
LBFGS:    6 00:50:02     -182.087333        0.726929
LBFGS:    7 00:50:02     -182.118180        0.637846
LBFGS:    8 00:50:02     -182.136423        0.306306
LBFGS:    9 00:50:02     -182.143288        0.168359
LBFGS:   10 00:50:03     -182.148310        0.209468
LBFGS:   11 00:50:03     -182.153332        0.242485
LBFGS:   12 00:50:03     -182.160019        0.266421
LBFGS:   13 00:50:03     -182.165339        0.221771
LBFGS:   14 00:50:03     -182.170392        0.233028
LBFGS:   15 00:50:03     -182.176741        0.319021
LBFGS:   16 00:50:03     -182.183016        0.271902
LBFGS:   17 00:50:03     -18

np.True_